# Rate-Limited Querying of Github's GraphQL API

In [56]:
import datetime
import time

import pandas as pd
import requests

In [2]:
file = open("/home/joseph/graphql_token.txt", "r")
api_token = file.read().strip()

url = "https://api.github.com/graphql"
headers = {"Authorization": "token %s" % api_token}

In [3]:
def query(json: str) -> dict:
    r = requests.post(url=url, json=json, headers=headers)
    return r.json()

### Sample Query

The data is returned in a JSON structure, as a `str` type object. 

In [4]:
json = {
    "query": "{ viewer { repositories(first: 1) { totalCount pageInfo { hasNextPage endCursor } edges { node { name } } } } }"
}

query(json)

{'data': {'viewer': {'repositories': {'totalCount': 38,
    'pageInfo': {'hasNextPage': True, 'endCursor': 'Y3Vyc29yOnYyOpHOAvjC2Q=='},
    'edges': [{'node': {'name': 'Yendors-Analysis'}}]}}}}

This query returns the remaining number of nodes queries, as well as the when the limit will be reset. This data can be passed into a pandas `DataFrame` using `pd.read_json()`, and from there the remaining limit and reset time can be parsed to allow for rate-limited programmatic scraping of Github's GraphQL API.

In [5]:
query_text = """query {
  viewer {
    login
  }
  rateLimit {
    limit
    cost
    remaining
    resetAt
  }
}"""

json = {"query": query_text}
query(json)

{'data': {'viewer': {'login': 'beverast'},
  'rateLimit': {'limit': 5000,
   'cost': 1,
   'remaining': 4998,
   'resetAt': '2019-08-28T17:54:18Z'}}}

### Ingest GraphQL Responses Into a DataFrame
**1. Query the endpoint, ingest as a DataFrame from JSON**

In [6]:
limit_df = pd.DataFrame(query(json))
limit_df = limit_df.reset_index()

In [7]:
limit_df.head()

,index,data
0,rateLimit,"{'limit': 5000, 'cost': 1, 'remaining': 4997, ..."
1,viewer,{'login': 'beverast'}


**2. Create columns for the necessary data: `remaining` and `resetAt`**

In [8]:
limit_df["remaining"] = limit_df.iloc[0][1]['remaining']
limit_df["resetAt"] = pd.Timestamp(limit_df.iloc[0][1]['resetAt'])

**3. Drop unnecessary `viewer` data**

In [9]:
limit_df = limit_df.drop(axis=1, index=1)

In [10]:
limit_df.head()

,index,data,remaining,resetAt
0,rateLimit,"{'limit': 5000, 'cost': 1, 'remaining': 4997, ...",4997,2019-08-28 17:54:18+00:00


### Automating Queries

In [11]:
def update_limit() -> pd.DataFrame:
    """ Returns a DataFrame of current GraphQL API query limits."""
    query_text = """query {
                      viewer {
                        login
                      }
                      rateLimit {
                        limit
                        cost
                        remaining
                        resetAt
                      }
                    }"""

    json = {"query": query_text}
    query(json)
    df = pd.DataFrame(query(json)).reset_index()
    
    df["remaining"] = df.iloc[0][1]['remaining']
    df["resetAt"] = pd.Timestamp(df.iloc[0][1]['resetAt'])
    df = df.drop(axis=1, index=1)
    
    return df


def get_remaining(df: pd.DataFrame) -> int:
    """Returns the remaining count of node queries."""
    return df['remaining'][0].astype(int)


def get_resetAt(df: pd.DataFrame) -> pd.Timestamp:
    """Retruns the time (UTC) at which the remaining queries will be reset to 5000."""
    return pd.Timestamp(df['resetAt'][0])


def is_resetAt_reached(df: pd.DataFrame) -> bool:
    """Returns True or False if the resetAt time has been reached."""    
    resetAt = get_resetAt(df)
    return pd.Timestamp.now(tz='UTC') > resetAt

In [13]:
limit_df = update_limit()
remaining = get_remaining(limit_df)
resetAt = get_resetAt(limit_df)

remaining, resetAt

(4995, Timestamp('2019-08-28 17:54:18+0000', tz='UTC'))

#### The following two code cells are from [this notebook](https://github.com/labs15-github-commit/data-science/blob/patrick/toDataFrameFunction.ipynb) by Patrick Wilky.

In [14]:
def better_df(x):
    """
    This function will take the raw query results and turn it into a pretty dataframe.
    """
    data = x.get('data').get('search').get('nodes')
    
    copy = pd.DataFrame(data)
    
    templist = []
    templist2 = []
    
    for i in copy.comments:
        templist.append(i.get('totalCount'))
    copy['commentCount'] = templist
    
    templist.clear()
    for i in copy.comments:
        templist.append(i.get('edges'))
    copy['comments'] = templist
    
    templist.clear()
    for i in range(len(copy.comments)):
        templist2 = []
        if (copy.commentCount[i]==0):
            templist.append(templist2)
        else:
            for o in copy.comments[i]:
                templist2.append(o.get('node'))
                if (len(templist2)==copy.commentCount[i]):
                    templist.append(templist2)                   
    copy['comments'] = templist
    
    templist.clear()
    for i in copy.author:
        templist.append(i.get('company'))
    copy['company'] = templist 
    
    templist.clear()
    for i in copy.author:
        templist.append(i.get('login'))
    copy['author'] = templist   
    
    templist.clear()
    for i in copy.files:
        templist.append(i.get('totalCount'))
    copy['filesCommited'] = templist
    
    
    templist.clear()
    for i in copy.mergedBy:
        if (i == None):
            templist.append(None)
        else:
            templist.append(i.get('login'))
    copy['mergedBy'] = templist
    
    copy = copy.drop(columns='files')
    
    return copy

In [74]:
pull_request_query = """
{
  search(first:100, query:"repo:pandas-dev/pandas created:2018-08-01..2018-08-12 type:pr", type:ISSUE) {
    nodes {
      ... on PullRequest {
        createdAt
        updatedAt
        title
        mergedBy {
          login
        }
        authorAssociation
        author {
          login
          ... on User {
            company
          }
        }
        files {
          totalCount
        }
        state
        resourcePath
        bodyText
        comments(first: 50) {
          totalCount
          edges {
            node {
              authorAssociation
              author{
                login
              }
              bodyText
            }
          }
        }
      }
    }
  }
}
"""

In [72]:
json = {"query": pull_request_query}
response = query(json)
pull_request_df = better_df(response)

In [73]:
pull_request_df.head()

,author,authorAssociation,bodyText,comments,createdAt,mergedBy,resourcePath,state,title,updatedAt,commentCount,company,filesCommited
0,jseabold,CONTRIBUTOR,2 issues for DataFrame.to_csv\n1 ) If header a...,"[{'authorAssociation': 'MEMBER', 'author': {'l...",2011-09-17T17:56:07Z,None,/pandas-dev/pandas/pull/151,CLOSED,ENH: improve DataFrame read_csv / to_csv for I...,2014-06-19T05:29:22Z,1,Civis Analytics,3
1,MLnick,NONE,"Hi Wes\nFirstly, congrats on such an amazing p...","[{'authorAssociation': 'MEMBER', 'author': {'l...",2011-09-16T09:32:28Z,None,/pandas-dev/pandas/pull/146,CLOSED,Minor change to csv reading,2014-06-16T01:22:51Z,1,IBM @CODAIT,1
2,jseabold,CONTRIBUTOR,More low-hanging fruit. Anywhere the indices h...,"[{'authorAssociation': 'MEMBER', 'author': {'l...",2011-09-14T23:59:36Z,None,/pandas-dev/pandas/pull/142,CLOSED,ENH: Allow unstacking by level name,2014-06-14T02:28:14Z,1,Civis Analytics,1
3,jseabold,CONTRIBUTOR,Trying to implement the easy items on my wishl...,"[{'authorAssociation': 'MEMBER', 'author': {'l...",2011-09-14T23:17:01Z,None,/pandas-dev/pandas/pull/141,CLOSED,ENH: Allow to sort on index level by name,2014-06-16T22:03:18Z,1,Civis Analytics,1
4,jseabold,CONTRIBUTOR,The convenience function I've been using. Don'...,"[{'authorAssociation': 'MEMBER', 'author': {'l...",2011-09-14T21:43:18Z,None,/pandas-dev/pandas/pull/140,CLOSED,ENH: Add panel_index convenience function,2014-08-02T09:06:16Z,1,Civis Analytics,1


**1. Create GraphQL query to get a repo's `createdAt` and `updatedAt` info**

In [18]:
# From Dustin Yang's graphql.ipynb
repo_query = """
{
  repositoryOwner(login: "pandas-dev") {
    id
    login
    repository(name: "pandas") {
      id
      name
      createdAt
      updatedAt
      description
    }
  }
}
"""

In [19]:
json = {"query": repo_query}
query_results = query(json)

**2. Ingest as a DataFrame**

In [20]:
query_dict = query_results['data']['repositoryOwner']['repository']
repo_df = pd.DataFrame.from_dict(query_dict, orient='index').T
repo_df

,id,name,createdAt,updatedAt,description
0,MDEwOlJlcG9zaXRvcnk4NTgxMjc=,pandas,2010-08-24T01:37:33Z,2019-08-28T16:52:02Z,Flexible and powerful data analysis / manipula...


**3. Convert to `pd.Timestamp`**

In [21]:
repo_df['createdAt'] = pd.Timestamp(repo_df['createdAt'][0])
repo_df['updatedAt'] = pd.Timestamp(repo_df['updatedAt'][0])

repo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 5 columns):
id             1 non-null object
name           1 non-null object
createdAt      1 non-null datetime64[ns, UTC]
updatedAt      1 non-null datetime64[ns, UTC]
description    1 non-null object
dtypes: datetime64[ns, UTC](2), object(3)
memory usage: 120.0+ bytes


**4. Generate date intervals for querying**

In [22]:
repo_df['updatedAt'][0] - repo_df['createdAt'][0]

Timedelta('3291 days 15:14:29')

In [23]:
repo_months = pd.date_range(start=repo_df['createdAt'][0], end=repo_df['updatedAt'][0], freq='M', tz='UTC')
repo_months

DatetimeIndex(['2010-08-31 01:37:33+00:00', '2010-09-30 01:37:33+00:00',
               '2010-10-31 01:37:33+00:00', '2010-11-30 01:37:33+00:00',
               '2010-12-31 01:37:33+00:00', '2011-01-31 01:37:33+00:00',
               '2011-02-28 01:37:33+00:00', '2011-03-31 01:37:33+00:00',
               '2011-04-30 01:37:33+00:00', '2011-05-31 01:37:33+00:00',
               ...
               '2018-10-31 01:37:33+00:00', '2018-11-30 01:37:33+00:00',
               '2018-12-31 01:37:33+00:00', '2019-01-31 01:37:33+00:00',
               '2019-02-28 01:37:33+00:00', '2019-03-31 01:37:33+00:00',
               '2019-04-30 01:37:33+00:00', '2019-05-31 01:37:33+00:00',
               '2019-06-30 01:37:33+00:00', '2019-07-31 01:37:33+00:00'],
              dtype='datetime64[ns, UTC]', length=108, freq='M')

**5. Insert these `DatetimeIndex` values into queries**

In [24]:
from string import Template

pr_search_token = Template("""
{
  search(first:100, query:"repo:pandas-dev/pandas created:$date1..$date2 type:pr", type:ISSUE) {""")

pr_query_wo_search = """
    nodes {
      ... on PullRequest {
        createdAt
        updatedAt
        title
        mergedBy {
          login
        }
        authorAssociation
        author {
          login
          ... on User {
            company
          }
        }
        files {
          totalCount
        }
        state
        resourcePath
        bodyText
        comments(first: 50) {
          totalCount
          edges {
            node {
              authorAssociation
              author{
                login
              }
              bodyText
            }
          }
        }
      }
    }
  }
}
"""

In [25]:
str(repo_months[0].date()), str(repo_months[1].date())

('2010-08-31', '2010-09-30')

In [26]:
month_pairs = list(zip(repo_months,repo_months[1:]))
month_pairs[:5]

[(Timestamp('2010-08-31 01:37:33+0000', tz='UTC', freq='M'),
  Timestamp('2010-09-30 01:37:33+0000', tz='UTC', freq='M')),
 (Timestamp('2010-09-30 01:37:33+0000', tz='UTC', freq='M'),
  Timestamp('2010-10-31 01:37:33+0000', tz='UTC', freq='M')),
 (Timestamp('2010-10-31 01:37:33+0000', tz='UTC', freq='M'),
  Timestamp('2010-11-30 01:37:33+0000', tz='UTC', freq='M')),
 (Timestamp('2010-11-30 01:37:33+0000', tz='UTC', freq='M'),
  Timestamp('2010-12-31 01:37:33+0000', tz='UTC', freq='M')),
 (Timestamp('2010-12-31 01:37:33+0000', tz='UTC', freq='M'),
  Timestamp('2011-01-31 01:37:33+0000', tz='UTC', freq='M'))]

In [27]:
pr_query_list = []
for pair in month_pairs:
    date1 = str(pair[0].date())
    date2 = str(pair[1].date())
    pr_query = pr_search_token.substitute(date1=date1, date2=date2)
    pr_query_list.append(pr_query)

pr_query_list[:10]

['\n{\n  search(first:100, query:"repo:pandas-dev/pandas created:2010-08-31..2010-09-30 type:pr", type:ISSUE) {',
 '\n{\n  search(first:100, query:"repo:pandas-dev/pandas created:2010-09-30..2010-10-31 type:pr", type:ISSUE) {',
 '\n{\n  search(first:100, query:"repo:pandas-dev/pandas created:2010-10-31..2010-11-30 type:pr", type:ISSUE) {',
 '\n{\n  search(first:100, query:"repo:pandas-dev/pandas created:2010-11-30..2010-12-31 type:pr", type:ISSUE) {',
 '\n{\n  search(first:100, query:"repo:pandas-dev/pandas created:2010-12-31..2011-01-31 type:pr", type:ISSUE) {',
 '\n{\n  search(first:100, query:"repo:pandas-dev/pandas created:2011-01-31..2011-02-28 type:pr", type:ISSUE) {',
 '\n{\n  search(first:100, query:"repo:pandas-dev/pandas created:2011-02-28..2011-03-31 type:pr", type:ISSUE) {',
 '\n{\n  search(first:100, query:"repo:pandas-dev/pandas created:2011-03-31..2011-04-30 type:pr", type:ISSUE) {',
 '\n{\n  search(first:100, query:"repo:pandas-dev/pandas created:2011-04-30..2011-05-31 

In [28]:
test_query = pr_query_list[99]+pr_query_wo_search
print(test_query)


{
  search(first:100, query:"repo:pandas-dev/pandas created:2018-11-30..2018-12-31 type:pr", type:ISSUE) {
    nodes {
      ... on PullRequest {
        createdAt
        updatedAt
        title
        mergedBy {
          login
        }
        authorAssociation
        author {
          login
          ... on User {
            company
          }
        }
        files {
          totalCount
        }
        state
        resourcePath
        bodyText
        comments(first: 50) {
          totalCount
          edges {
            node {
              authorAssociation
              author{
                login
              }
              bodyText
            }
          }
        }
      }
    }
  }
}



In [31]:
json = {"query": test_query}
response = query(json)
pr_test_query = better_df(response)

In [32]:
pr_test_query.head()

,author,authorAssociation,bodyText,comments,createdAt,mergedBy,resourcePath,state,title,updatedAt,commentCount,company,filesCommited
0,TomAugspurger,CONTRIBUTOR,,"[{'authorAssociation': 'NONE', 'author': {'log...",2018-12-31T21:06:34Z,jreback,/pandas-dev/pandas/pull/24524,MERGED,Fixed PeriodArray._time_shift positional argument,2019-01-02T20:18:03Z,3,@ContinuumIO,1
1,datapythonista,MEMBER,xref #22743\nWe already had an option --warnin...,"[{'authorAssociation': 'NONE', 'author': {'log...",2018-12-31T20:49:21Z,jreback,/pandas-dev/pandas/pull/24523,MERGED,DOC: Make sphinx fail the build when --warning...,2018-12-31T23:17:51Z,3,None,2
2,simonjayhawkins,MEMBER,xref #22715 (comment),"[{'authorAssociation': 'NONE', 'author': {'log...",2018-12-31T20:47:18Z,jreback,/pandas-dev/pandas/pull/24522,MERGED,REF/TST: use monkeypatch in mock clipboard fix...,2019-01-01T21:01:59Z,3,None,1
3,reidy-p,CONTRIBUTOR,… tests\n\n Progress towards #24471\n tests ad...,"[{'authorAssociation': 'NONE', 'author': {'log...",2018-12-31T20:09:54Z,jreback,/pandas-dev/pandas/pull/24521,MERGED,ENH: Add sort parameter to set operations for ...,2019-01-27T01:19:41Z,11,None,15
4,gfyoung,MEMBER,Removes the DeprecationWarning raised by dateu...,"[{'authorAssociation': 'NONE', 'author': {'log...",2018-12-31T19:41:22Z,jreback,/pandas-dev/pandas/pull/24520,MERGED,MAINT: Port _timelex in codebase,2019-01-01T18:41:52Z,5,None,4


### Functionalize, Write as a Loop

In [90]:
def auto_query(repository_owner: str, repository_name: str) -> pd.DataFrame:
    """
    Given a repository_owner and a repository name, return a DataFrame of pull request data
    for the entire history of the repo.
    """
    
    # Get API query limits
    limit_df = update_limit()
    remaining = get_remaining(limit_df)
    resetAt = get_resetAt(limit_df)

    # Query template for getting repo info
    repo_query = Template("""
    {
      repositoryOwner(login: "$repo_owner") {
        id
        login
        repository(name: "$repo_name") {
          id
          name
          createdAt
          updatedAt
          description
        }
      }
    }
    """)
    
    # Customize template for given arguments
    repo_query = repo_query.substitute(repo_owner=repository_owner, repo_name=repository_name)

    # Perform the GraphQL query
    json = {"query": repo_query}
    query_results = query(json)
    
    # Convert to DataFrame
    query_dict = query_results['data']['repositoryOwner']['repository']
    repo_df = pd.DataFrame.from_dict(query_dict, orient='index').T
    
    # Type conversion
    repo_df['createdAt'] = pd.Timestamp(repo_df['createdAt'][0])
    repo_df['updatedAt'] = pd.Timestamp(repo_df['updatedAt'][0])

    # Get dates for generated queries
    repo_months = pd.date_range(start=repo_df['createdAt'][0], end=repo_df['updatedAt'][0], freq='M', tz='UTC')
    month_pairs = list(zip(repo_months,repo_months[1:]))

    # Generate list of first half of formatted queries
    pr_query_list = []
    for pair in month_pairs[60:]:
        date1 = str(pair[0].date())
        date2 = str(pair[1].date())
        pr_query = pr_search_token.substitute(date1=date1, date2=date2)
        pr_query_list.append(pr_query)

    # Generate DataFrame for all queries
    total_repo_issues_df = pd.DataFrame()
    
    for search_token in pr_query_list:
        concat_query = search_token+pr_query_wo_search
        print(concat_query)    # DEBUG STATEMENT
        json = {"query": concat_query}
        query_response = query(json)
        print(query_response)    # DEBUG STATEMENT
        pr_query = better_df(query_response)
        
        # Check query limit
        limit_df = update_limit()
        remaining = get_remaining(limit_df)
        resetAt = get_resetAt(limit_df)
        
        print(total_repo_issues_df.tail())    # DEBUG STATEMENT
        
        if remaining > 20:
            print('Query limit remaining: ', remaining)
            total_repo_issues_df = pd.concat([total_repo_issues_df, pr_query])
            continue
        else:
            print('Query limit reached. Query limit resets at: ', resetAt)
            # Make loop wait until Query limit is reset: time.sleep(secs)
            while resetAt.time() > datetime.datetime.utcnow().time():
                time.wait(600)
                limit_df = update_limit()
                resetAt = get_resetAt(limit_df)
            else:
                break
    
    return total_repo_issues_df

In [91]:
auto_query("pandas-dev", "pandas")


{
  search(first:100, query:"repo:pandas-dev/pandas created:2015-08-31..2015-09-30 type:pr", type:ISSUE) {
    nodes {
      ... on PullRequest {
        createdAt
        updatedAt
        title
        mergedBy {
          login
        }
        authorAssociation
        author {
          login
          ... on User {
            company
          }
        }
        files {
          totalCount
        }
        state
        resourcePath
        bodyText
        comments(first: 50) {
          totalCount
          edges {
            node {
              authorAssociation
              author{
                login
              }
              bodyText
            }
          }
        }
      }
    }
  }
}

{'data': {'search': {'nodes': [{'createdAt': '2015-09-30T23:23:51Z', 'updatedAt': '2015-11-20T14:51:30Z', 'title': 'BUG: GH11206 where pd.isnull did not consider numpy NaT null', 'mergedBy': {'login': 'jreback'}, 'authorAssociation': 'CONTRIBUTOR', 'author': {'login': 'kawo

AttributeError: 'NoneType' object has no attribute 'get'